# **Canais Quânticos - dois nós conectados por um canal quântico** 

Neste notebook praticaremos como conectar nós uns aos outros para que os qubits possam ser enviados entre eles. Até agora armazenamos qubits, aplicamos operações e os medimos. Agora daremos um passo adiante e também enviaremos qubits entre diferentes locais. <br> <br>

Em primeiro lugar, agora deixamos de trabalhar diretamente com memórias quânticas ou processadores quânticos e passamos a trabalhar com nós que possuem processadores quânticos dentro deles. Os nós podem ter portas de entrada e saída que nos permitem conectar diferentes nós entre si por meio de canais clássicos ou quânticos.

In [1]:
import netsquid as ns
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.qprogram import QuantumProgram
from netsquid.nodes import Node
from netsquid.nodes.connections import Connection
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qchannel import QuantumChannel
import netsquid.components.instructions as instr
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.protocols.protocol import Signals
import pydynaa


## <span style = "color: black">**Exemplo 1: Dois nós conectados por um canal quântico**</span>

Em nosso primeiro exemplo, conectaremos dois nós por um canal quântico e usaremos esse canal para enviar qubits de um nó para outro.
Nós começamos definindo as instruções físicas e inicializando dois processadores quânticos com uma posição de memória cada.

In [2]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1),
    PhysicalInstruction(instr.INSTR_X, duration=1)]
qproc1a = QuantumProcessor(name="ExampleQProc1a" , num_positions =1, phys_instructions = phys_instructions1)
qproc1b = QuantumProcessor(name="ExampleQProc1b" , num_positions =1, phys_instructions = phys_instructions1)

<br> <br> Cada processador será atribuído a um nó. Os dois nós estarão fisicamente separados um do outro. Para fazer isso, inicializaremos os dois nós com portas para comunicação de entrada (qin) e saída (qout). <br> <br>

- <span style = "color: orange"> class netsquid.nodes.Node(name, ID= None, qmemory = None, port_names=None) </span>

In [3]:
node1a = Node(name="ExampleNode1a", qmemory=qproc1a, port_names=['qin_1a','qout_1a'])
node1b = Node(name="ExampleNode1b", qmemory=qproc1b, port_names=['qin_1b','qout_1b'])

<br> <br> Nós também inicializamos um canal quântico que usaremos para conectar os dois nós. <br> <br>
- <span style = "color: orange"> class netsquid.components.qchanel.QuantumChannel(name, delay=0, lenght=0, models=None, transmit_empty_items=False, properties=None, **kwargs) </span>

In [4]:
qchannell = QuantumChannel(name="QChannel_1a21b")

<br> <br> Nosso objetivo é:
- Inicializar um qubit no slot 0 em node1a;
- Enviar o qubit através do canal para node1b;
- Armazene o qubit no slot 0 no node1b. <br> <br>

Para fazer isso, precisamos conectar os nós por um canal quântico. <br> <br>
Vamos fazer isso da seguinte forma: <br> <br>

- Enviando a saída da posição de memória 0 do nó 1a para a saída do nó 1a ;
- Enviando a entrada do nó 1b upara a entrada da posição de memória 0 no nó 1b;
- Conectando a saída do nó 1a com a entrada do canal quântico;
- Conectando a saída do canal quântico com a entrada do nó 1b.

Desta forma podemos enviar um qubit do nó 1a para o nó 1b.

<img src = nodes.png>

In [5]:
node1a.qmemory.ports['qout0'].forward_output(node1a.ports['qout_1a'])
node1b.ports['qin_1b'].forward_input(node1b.qmemory.ports['qin0'])

In [6]:
node1a.ports['qout_1a'].connect(qchannell.ports["send"])
node1b.ports['qin_1b'].connect(qchannell.ports["recv"])

<br><br>Inicializando um qubit no slot 0 no node1a.

In [7]:
node1a.qmemory.execute_instruction(instr.INSTR_INIT, [0]) # Agendando inicialização do qubit
ns.sim_run() # Rodando o próximo evento agendado
ns.sim_time() # Checando o tempo atual

3.0

Espiando a memória, vemos que existe um qubit.

In [8]:
node1a.qmemory.peek(positions=[0])

[Qubit('QS#0-0')]

<br> <br> Em seguida, nós queremos enviar um qubit através do canal para o node1b. Para fazer isso, removemos (ou usamos pop) o qubit da memória e o colocamos na porta de saída da memória. <br> <br>

<span style = "color:orange"> pop(self, positions, noise_positions = none, skip_noise=False, check_positions=True, meta_data = None, positional_qout=False) </span>

In [9]:
qubit = node1a.qmemory.pop(positions=[0]) # removendo o qubit da posição 0
node1a.qmemory.ports["qout0"].tx_output(qubit)

<br> <br> Espiando agora, nós vemos que não há nenhum qubit na memória.

In [10]:
node1a.qmemory.peek(positions=[0])

[None]

Espiando a memória no node1b, nós vemos que o qubit não chegou ainda.

In [11]:
node1b.qmemory.peek(positions=[0])

[None]

Para realmente mover o qubit através do canal, precisamos executar o simulador que executará o evento de transmissão do qubit

In [12]:
ns.sim_run()
ns.sim_time()

3.0

Verificando o tempo de simulação, vemos que não passou nenhum tempo enquanto o qubit foi enviado pelo canal. Isso ocorre porque na verdade não contabilizamos a duração desse evento. No momento, a transmissão é instantânea. (Nota: poderíamos ter modelado, por exemplo, os qubits como fótons viajando através de uma fibra. Faremos isso mais tarde).

Espiando o node1b, vemos que o qubit chegou e agora está armazenado na memória do node1b.

In [13]:
node1b.qmemory.peek(positions=[0])

[Qubit('QS#0-0')]

## <span style="color:black">**Sugestão de Prática:**


Crie um novo exemplo no qual você permite que o qubit vá e volte entre os dois nós usando dois canais quânticos.